# 君維的FB抓取授眾程式修改+註解

In [16]:
# -*- coding: utf-8 -*-
import requests
import json
import re
#查詢的東西
q = '吞劍'
#性別
gender = 1
ageMin = 18
ageMax = 65

###找js裡面targetingsearch裡面response是顯示idxxxxxxxx的headers複製Request URL:中間有一段token當token1
token1 = 'EAAI4BG12pyIBAIKfnlvJDbDeebUUOl5LYb8tzOgLVqMOZCCKr7QGKEJmZBFAuZCt01tdy08Lqm6IK9rbZCJlmZCPG2mcWyT68f1jeNOzHlzZAL9cwS9lzOWZAtk46H8hWfCjqGR2gAnqn91Y9Lhlk7iaZALwCTkOWzyEc41195A8gAZDZD'
###找js裡面reachestimate裡面response是顯示正確授眾users人數的headers複製Request URL:中間有一段token當token2
token2 = 'EAAI4BG12pyIBAIKfnlvJDbDeebUUOl5LYb8tzOgLVqMOZCCKr7QGKEJmZBFAuZCt01tdy08Lqm6IK9rbZCJlmZCPG2mcWyT68f1jeNOzHlzZAL9cwS9lzOWZAtk46H8hWfCjqGR2gAnqn91Y9Lhlk7iaZALwCTkOWzyEc41195A8gAZDZD'
header ={
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36'
}
#為符合fb的格式而建立一個空list, 用來把性別放進去
genderShell = []
genderDict = {0: '所有', 1: '男性', 2: '女性'}
#送出查詢請求
###找js裡面targetingsearch裡面response是顯示idxxxxxxxx的headers複製Request URL:
###要把act_和accountId=改成自己的id
url = 'https://graph.facebook.com/v2.5/act_10152010794069053/targetingsearch?_reqName=path%3A%2Fact_135203665%2Ftargetingsearch&_reqSrc=AdsDaoGraphDataProvider&access_token={}&accountId=10152010794069053&endpoint=%2Fact_135203665%2Ftargetingsearch&locale=zh_TW&method=get&pretty=0&q={}&suppress_http_code=1'
res = requests.get(url.format(token1, q), headers = header)
#實際比對到的東西
actualName = json.loads(res.text)['data'][0]['name']
#.encode('utf-8')
#從查詢的回應中取得第一筆也就是比對最接近的結果id
qid = json.loads(res.text)['data'][0]['id']
#建立查詢字典
interest = {"geo_locations":{"countries":["TW"],"location_types":["home","recent"]},"age_min":"","age_max":"","page_types":["mobilefeed","desktopfeed"],"flexible_spec":[{"interests":[{"id":"","name":""}]}]}
#將查詢條件寫入字典
interest['flexible_spec'][0]['interests'][0]['id'] = qid
interest['flexible_spec'][0]['interests'][0]['name'] = q
interest['age_min'] = ageMin
interest['age_max'] = ageMax
if gender != 0:
    genderShell.append(gender)
    interest['genders'] = genderShell
#準備送人數估計的請求，先把targeting_spec挖空等著把前面建的字典塞進去
###找js裡面reachestimate裡面response是顯示正確授眾users人數的headers複製Request URL:
###要把act_改成自己的id
sizereq = 'https://graph.facebook.com/v2.4/act_10152010794069053/reachestimate?_reqName=adaccount/reachestimate&_reqSrc=AdsCFAudienceEstimateUtils&access_token={}&bid_for=["conversion"]&callback=__globalCallbacks.f30157a059eae7c&currency=TWD&locale=zh_TW&method=get&pretty=0&targeting_spec={}'

#用dumps把字典轉為字串丟進去洞裡面，送出請求
resSize = requests.get(sizereq.format(token2, json.dumps(interest)), headers = header)
#萃取回應中json格式的內容
result = re.search('{.*}',resSize.text)
finalResult = json.loads(result.group(0))
#取得人數
users = finalResult['users']
print ('%d-%d歲族群中%s的「%s」潛在喜愛者有%s人' % (ageMin, ageMax, genderDict[gender], actualName, users))

18-65歲族群中所有的「吞劍」潛在喜愛者有1100人


# 抓取露天小項

In [1]:
import requests
from bs4 import BeautifulSoup

headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


res = requests.get('http://class.ruten.com.tw/category/' , headers=headers)
soup = BeautifulSoup(res.text, "lxml")
#soup = BeautifulSoup(res.text, "lxml", from_encoding="GB18030")

#原始資料內含@ 取代
q=[]
for ens in soup.select('div.rt-grid-1-5'):          
    q.append((ens.select('a')[0].text).replace('@','').encode('latin-1').decode('utf-8'))
print (q)


[' 筆記型電腦 ', ' 筆記型電腦專用配件 ', ' 平板電腦 ', ' 桌上型電腦 ', ' 電腦周邊設備 ', ' 電腦周邊耗材 ', ' 電腦軟體 ', ' 租賃、出租 ', ' 電腦零組件 ', ' 電子零件、材料 ', ' 電子字典 ', ' 電腦保養 ', ' 網路設備 ', ' 電子儀器、測量 ', ' 貿易製造 ', ' 化工、容器材料、儀器 ', ' 線材、轉接頭 ', ' 高科技、機器人 ', ' GPS / 穿戴 / 導航 ', ' 電腦、網路諮詢 ', ' 代買、海外代購 ', ' T恤 ', ' 襯衫 ', ' 背心 ', ' 小可愛 ', ' 有領休閒衫、POLO衫 ', ' 毛衣 ', ' 外套 ', ' 2件式上衣 ', ' 中國風上衣 ', ' 針織衫 ', ' 棉麻上衣 ', ' 絲紗上衣 ', ' 毛料上衣 ', ' 裙子 ', ' 褲子 ', ' 洋裝 ', ' 套裝 ', ' 表演服、道具服 ', ' 禮服、晚宴服 ', ' 大尺寸 ', ' 品牌專區 ', ' 國際代購/代買 ', ' 其他 ', ' 女性各類包包 ', ' 女性首飾配件 ', ' 香水 ', ' T恤 ', ' 襯衫 ', ' POLO衫 ', ' 針織衫 ', ' 背心 ', ' 毛衣 ', ' 外套 ', ' 褲子 ', ' 西裝 ', ' 內衣褲 ', ' 大尺寸 ', ' 表演、道具服 ', ' 其他男裝 ', ' 背包/公事包 ', ' 首飾配件 ', ' 打火機 ', ' 鑰匙圈 ', ' 皮夾 ', ' 領帶、吊帶 ', ' 睡衣 ', ' 皮飾、皮帶 ', ' 帽子 ', ' 手套 ', ' 假髮 ', ' 圍巾、手帕 ', ' 襪子 ', ' 其他 ', ' 男運動服 ', ' 男童裝 ', ' 男錶 ', ' 男鞋 ', ' 男士保養 ', ' 其他隨身配件 ', ' 國際代購/代買 ', ' 可動玩偶 ', ' 轉蛋食玩 ', ' PVC 人偶 ', ' 卡漫收藏 ', ' 塑膠組裝模型 ', ' 遙控模型、玩具 ', ' 日本景品 ', ' 金屬模型 ', ' 鐵道模型 ', ' GK模型 ', ' 兒童、益智遊戲 ', ' 連鎖店玩具 ', ' 洋娃娃與配件 ', ' 絨毛、填充玩偶 ', ' 紙牌遊戲 ', ' 魔術道具 

# FB授眾+露天 (寫入txt檔,用逗號分隔)

In [21]:
# -*- coding: utf-8 -*-
import requests
import json
import re
#查詢的東西

###上面已抓露天分類小項做出興趣的list q
#q = '吞劍'

#性別
gender = [0,1,2]
ageMin = [13,16,21,26,31,36,41,51,56,61]
ageMax = [15,20,25,30,35,40,45,55,60,65]


with open('fb授眾word.txt', 'a') as f:
    ###  k迴圈控制興趣
    for k in range(0,len(q)):
    #for k in range(0,1):

        ###找js裡面targetingsearch裡面response是顯示idxxxxxxxx的headers複製Request URL:中間有一段token當token1
        token1 = 'EAAI4BG12pyIBAAfmQizBZC3Uh7RzUnjVcrQpDyjcDfQm9Hu5kRd4nx8kFWfo5R5tQoNovbRBqSh4JYwZANmxQZAVVKFu0VwiY6UlLRm9ZB840WSo5ksZCYgPFODxFDyRZB4MsmYErITHAVYGd7XE0QeIELtPrnU3FHiaPHKLE3ogZDZD'
        ###找js裡面reachestimate裡面response是顯示正確授眾users人數的headers複製Request URL:中間有一段token當token2
        token2 = 'EAAI4BG12pyIBAAfmQizBZC3Uh7RzUnjVcrQpDyjcDfQm9Hu5kRd4nx8kFWfo5R5tQoNovbRBqSh4JYwZANmxQZAVVKFu0VwiY6UlLRm9ZB840WSo5ksZCYgPFODxFDyRZB4MsmYErITHAVYGd7XE0QeIELtPrnU3FHiaPHKLE3ogZDZD'
        header ={
            'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36'
        }
        #為符合fb的格式而建立一個空list, 用來把性別放進去
        
        genderDict = {0: '所有', 1: '男性', 2: '女性'}
        #送出查詢請求
        ###找js裡面targetingsearch裡面response是顯示idxxxxxxxx的headers複製Request URL:
        ###要把act_和accountId=改成自己的id
        url = 'https://graph.facebook.com/v2.5/act_10152010794069053/targetingsearch?_reqName=path%3A%2Fact_135203665%2Ftargetingsearch&_reqSrc=AdsDaoGraphDataProvider&access_token={}&accountId=10152010794069053&endpoint=%2Fact_135203665%2Ftargetingsearch&locale=zh_TW&method=get&pretty=0&q={}&suppress_http_code=1'
        res = requests.get(url.format(token1, q[k]), headers = header)
        #實際比對到的東西
        ###因為露天的品項中會有無授眾的品項,遇到該品項則用if跳過
        #print ((json.loads(res.text)['data']))
        if (len(json.loads(res.text)['data'])==0):
            continue


        ###  j迴圈控制年齡
        for j in range(0,len(ageMin)):
            

            ###  i迴圈控制性別
            for i in range(0,len(gender)):
                
                ###genderShell空陣列要放迴圈內
                genderShell = []
                actualName = json.loads(res.text)['data'][0]['name']
                #.encode('utf-8')
                #從查詢的回應中取得第一筆也就是比對最接近的結果id
                qid = json.loads(res.text)['data'][0]['id']
                #建立查詢字典
                interest = {"geo_locations":{"countries":["TW"],"location_types":["home","recent"]},"age_min":"","age_max":"","page_types":["mobilefeed","desktopfeed"],"flexible_spec":[{"interests":[{"id":"","name":""}]}]}
                #將查詢條件寫入字典
                interest['flexible_spec'][0]['interests'][0]['id'] = qid
                interest['flexible_spec'][0]['interests'][0]['name'] = q[k]
                interest['age_min'] = ageMin[j]
                interest['age_max'] = ageMax[j]
                if i != 0:
                    genderShell.append(i)
                    interest['genders'] = genderShell
                #準備送人數估計的請求，先把targeting_spec挖空等著把前面建的字典塞進去
                ###找js裡面reachestimate裡面response是顯示正確授眾users人數的headers複製Request URL:
                ###要把act_改成自己的id
                sizereq = 'https://graph.facebook.com/v2.4/act_10152010794069053/reachestimate?_reqName=adaccount/reachestimate&_reqSrc=AdsCFAudienceEstimateUtils&access_token={}&bid_for=["conversion"]&callback=__globalCallbacks.f30157a059eae7c&currency=TWD&locale=zh_TW&method=get&pretty=0&targeting_spec={}'

                #用dumps把字典轉為字串丟進去洞裡面，送出請求
                resSize = requests.get(sizereq.format(token2, json.dumps(interest)), headers = header)
                #萃取回應中json格式的內容
                result = re.search('{.*}',resSize.text)
                finalResult = json.loads(result.group(0))
                #print (finalResult)
                #取得人數
                ###因為露天小項中有屬於"非興趣"項目 先用try exception處理
                try:
                    users = finalResult['users']
                    print ('%d-%d歲族群中%s的「%s」潛在喜愛者有%s人' % (ageMin[j], ageMax[j], genderDict[i], actualName, users))

                except:
                    continue
                #p = (ageMin[j]+','+ ageMax[j]+','+ genderDict[i]+','+actualName+','+users)
                p = (str(ageMin[j])+','+str(ageMax[j])+','+ str(genderDict[i])+','+str(actualName)+','+str(users)+ '\n')
                #print (type(ageMin[j]))
                f.write(p)
                

13-15歲族群中所有的「筆記型電腦」潛在喜愛者有21000人
13-15歲族群中男性的「筆記型電腦」潛在喜愛者有9000人
13-15歲族群中女性的「筆記型電腦」潛在喜愛者有12000人
16-20歲族群中所有的「筆記型電腦」潛在喜愛者有170000人
16-20歲族群中男性的「筆記型電腦」潛在喜愛者有80000人
16-20歲族群中女性的「筆記型電腦」潛在喜愛者有93000人
21-25歲族群中所有的「筆記型電腦」潛在喜愛者有210000人
21-25歲族群中男性的「筆記型電腦」潛在喜愛者有96000人
21-25歲族群中女性的「筆記型電腦」潛在喜愛者有110000人
26-30歲族群中所有的「筆記型電腦」潛在喜愛者有170000人
26-30歲族群中男性的「筆記型電腦」潛在喜愛者有74000人
26-30歲族群中女性的「筆記型電腦」潛在喜愛者有92000人
31-35歲族群中所有的「筆記型電腦」潛在喜愛者有140000人
31-35歲族群中男性的「筆記型電腦」潛在喜愛者有75000人
31-35歲族群中女性的「筆記型電腦」潛在喜愛者有70000人
36-40歲族群中所有的「筆記型電腦」潛在喜愛者有110000人
36-40歲族群中男性的「筆記型電腦」潛在喜愛者有66000人
36-40歲族群中女性的「筆記型電腦」潛在喜愛者有44000人
41-45歲族群中所有的「筆記型電腦」潛在喜愛者有79000人
41-45歲族群中男性的「筆記型電腦」潛在喜愛者有50000人
41-45歲族群中女性的「筆記型電腦」潛在喜愛者有29000人
51-55歲族群中所有的「筆記型電腦」潛在喜愛者有50000人
51-55歲族群中男性的「筆記型電腦」潛在喜愛者有28000人
51-55歲族群中女性的「筆記型電腦」潛在喜愛者有22000人
56-60歲族群中所有的「筆記型電腦」潛在喜愛者有9500人
56-60歲族群中男性的「筆記型電腦」潛在喜愛者有6400人
56-60歲族群中女性的「筆記型電腦」潛在喜愛者有3100人
61-65歲族群中所有的「筆記型電腦」潛在喜愛者有13000人
61-65歲族群中男性的「筆記型電腦」潛在喜愛者有8100人
61-65歲族群中女性的「筆記型電腦」潛在喜愛者有4600人
13-15歲族群中所有的「平板電腦」潛在喜愛者有23000人
13-15歲族群中

# 前昱的FB洞察報告修改+露天

In [4]:
import requests
import json
import re

###上面已抓露天分類小項做出興趣的list q
#查詢的東西
#q = '蛋糕'
gender = ['',1,2]
ageMin = [18,21,26,31,36,41,51,56,61]
ageMax = [20,25,30,35,40,45,55,60,65]


#每一個metric代表畫面的一個欄位# 9 是指教育程度內資料# 30廣告受眾總人數# 3男女分配
# 17生活型態# 27感情狀況# 15職稱
#metric='9'
metric = [9,27]
#要run哪幾種metric <---這邊應該要改dictionary比較好
ty=[ch,relationship]  


#給label<---這邊應該還是要改dictionary比較好
ch=[u'高中',u'大專院校',u'研究所']   
relationship=[u'單身',u'穩定交往中',u'已訂婚',u'已婚']
genderD = [u'所有', u'女性' ,u'男性']





###  k迴圈控制興趣
for k in range(0,len(q)):
#for k in range(0,1):

    #搜尋興趣代碼
    ###找js裡面targetingsearch裡面response是顯示idxxxxxxxx的headers複製Request URL:中間有一段token當token1
    token1 ='EAAJ6ne6FIaEBAPOFBGU44F2qZCm0tZBsgSZBc2qlwbFE09YSBHTqdhjxlFzqouhD8ZBY6AmoK8ekRcKvMICeUoyX3bQZBVH8lYXAftbyiIGUn3Vuqj8ffZCZAPY9BK18ywDwsAWFOrgrBMLHEZBZCf57gwBYOWO7S77sM5ZAE12fXp8QZDZD'

    header ={
    #     第二次要加cookie
        'cookie':'datr=tuHXVttlmgQXogp7W94k8zER; pl=n; lu=gA6tQ1t-Wu2Dp0aeK2xg1Bvg; c_user=675544052; fr=0Z80cbGNtwjoDvKjq.AWXf-lKTyvX8E9ocPnGmU21dg_4.BW1-HA.lK.FdH.0.0.AWWtN60T; xs=60%3AYDS33DIxPIyLcw%3A2%3A1463727842%3A14635; csm=2; s=Aa4PuHjjCQDPTf91.BXPrbi; sb=IAgHV-TBT735sVsMjsIpcnO6; act=1464607276124%2F5; p=-2; presence=EDvF3EtimeF1464607354EuserFA2675544052A2EstateFDutF1464607354915CEchFDp_5f675544052F2CC',
        'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    }

    url ='https://graph.facebook.com/v2.5/search?_reqName=path%3A%2Fsearch&_reqSrc=AdsDaoGraphDataProvider&access_token={}&endpoint=%2Fsearch&locale=zh_TW&method=get&pretty=0&q={}&type=adInterest'

    res = requests.get(url.format(token1, q[k]), headers = header)
    # print res.text

    #實際比對到的東西
    ###因為露天的品項中會有無授眾的品項,遇到該品項則用if跳過
    #print ((json.loads(res.text)['data']))
    if (len(json.loads(res.text)['data'])==0):
        continue

    ###  j迴圈控制年齡
    for j in range(0,len(ageMin)):

        ###  i迴圈控制性別
        for i in range(0,len(gender)):

            ###  n迴圈控制metric欄位
            for n in range(0,len(metric)):
                
                actualName = json.loads(res.text)['data'][0]['name']
                #.encode('utf-8')
                qid = json.loads(res.text)['data'][0]['id']
                #print (actualName)
                # print qid




                interest ='&age[0]={}&age[1]={}&country[0]=TW&gender={}&intere\
                sts[0]={}'.format(ageMin[j],ageMax[j],gender[i],qid)
                user1='admarket_id=6012041258962&logger_session_id=6c71d49613c2096478ad4e3ab9dcb66b5c2b764e&__user=675544052&__a=1&__dyn=aK4XK4GgW2KcCJDzo9EaUCbGujxPLFG5aw_UaEOVE98nwExeqq1yx-EK3eu4FoHByEjwSwzwh9UcU89o-2it1ScwuEgCGch8G9x24o&__req=1d'
                url2='https://www.facebook.com/ads/audience-insights/query/?dpr=1{}&metrics[0]={}&{}&__be=0&__pc=PHASED%3ADEFAULT&__rev=2358586'.format(interest,metric[n],user1)



                # print url2
                res2=requests.get(url2 ,headers = header)
                # print (res2.text)


                nu=0
                ###正規表示法去抓audience":{"total":後面的人數
                pattern = re.compile(r'"audience":{"total":(\d+)')
                

                
                for m in re.finditer(pattern,res2.text):
                    print (ageMin[j], ageMax[j], genderD[i], ty[n][nu] ,actualName ,m.group(1))
                    nu=nu+1


18 20 所有 高中 筆記型電腦 3176
18 20 所有 大專院校 筆記型電腦 671374
18 20 所有 研究所 筆記型電腦 399849
18 20 所有 單身 筆記型電腦 7627
18 20 所有 穩定交往中 筆記型電腦 31088
18 20 所有 已訂婚 筆記型電腦 158048
18 20 所有 已婚 筆記型電腦 401542
18 20 女性 高中 筆記型電腦 1240
18 20 女性 大專院校 筆記型電腦 331916
18 20 女性 研究所 筆記型電腦 193664
18 20 女性 單身 筆記型電腦 4806
18 20 女性 穩定交往中 筆記型電腦 22229
18 20 女性 已訂婚 筆記型電腦 94926
18 20 女性 已婚 筆記型電腦 177937
18 20 男性 高中 筆記型電腦 2198
18 20 男性 大專院校 筆記型電腦 338621
18 20 男性 研究所 筆記型電腦 206520
18 20 男性 單身 筆記型電腦 2516
18 20 男性 穩定交往中 筆記型電腦 8845
18 20 男性 已訂婚 筆記型電腦 63118
18 20 男性 已婚 筆記型電腦 223725
21 25 所有 高中 筆記型電腦 84474
21 25 所有 大專院校 筆記型電腦 1383414
21 25 所有 研究所 筆記型電腦 343615
21 25 所有 單身 筆記型電腦 21955
21 25 所有 穩定交往中 筆記型電腦 96159
21 25 所有 已訂婚 筆記型電腦 338840
21 25 所有 已婚 筆記型電腦 626430
21 25 女性 高中 筆記型電腦 33371
21 25 女性 大專院校 筆記型電腦 678138
21 25 女性 研究所 筆記型電腦 166296
21 25 女性 單身 筆記型電腦 12940
21 25 女性 穩定交往中 筆記型電腦 61872
21 25 女性 已訂婚 筆記型電腦 181131
21 25 女性 已婚 筆記型電腦 258456
21 25 男性 高中 筆記型電腦 49738
21 25 男性 大專院校 筆記型電腦 703515
21 25 男性 研究所 筆記型電腦 177548
21 25 男性 單身 筆記型電腦 8543
21 25 男性 穩定交往